## 아파트 실거래가 예측 프로젝트

### 목적
실제 한국의 부동산 데이터를 접해고 분석하는 과정을 통해,
어떠한 특징과 변수가 아파트 가격에 어떠한 영향을 주는지 분석해 본다.

데이터의 컬럼은 다음과 같습니다.
- index: 거래 번호
- sigungu: 시군구
- jibun: 지번 주소
- apt_name: 아파트 이름
- exclusive_use_area: 건물 내에서 개별적으로 소유자가 독점적으로 사용할 수 있는 공간
- transaction_year_month: 거래 연월
- transaction_day: 거래 날짜
- transaction_real_price: 실거래 가격
- floor: 층수
- year_of_completion: 건설 완료 연도

### Stage 1. 데이터 확인
프로젝트를 진행하기 앞서, 먼저 데이터를 확인해 보고 어떻게 분석할지 계획해보자.

1. 데이터 분석 전 준비

    [문제 1]

- CSV 파일을 데이터프레임 형식으로 읽어옵시다.

In [8]:
import os

print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\user\OneDrive\Upstage AI BootCamp\Machine Learning\ML 대회\Upstage_ML_Competition\Dacon\code


In [10]:
import pandas as pd

# 파일 읽기
train = pd.read_csv('../data/train.csv', encoding='utf-8')
test = pd.read_csv('../data/test.csv', encoding='utf-8')
day_care_center = pd.read_csv('../data/day_care_center.csv', encoding='utf-8')
park = pd.read_csv('../data/park.csv', encoding='utf-8')
submission = pd.read_csv('../data/submission.csv', encoding='utf-8')


2. 데이터 확인
    [문제 2]
- 빈칸을 채워 train의 인덱스 기준 상위 5개 데이터를 확인해 봅시다. 그리고 앞으로 어떻게 분석을 진행할지 생각해 봅시다.

[결과 해석]
- sigungu, jibun, apt_name 를 이용하면아파트의 위치를 파악할 수 있어 입지 분석을 할 수 있음.
- exclusive_use_area에는 아파트 가격에 중요한 요소인 면적 정보가 들어있음. 아파트 가격을 분석할 때, 면적에 따라 가격이 얼마나 차이 나는지 확인하는 것이 좋음.
- transaction_year_month, transaction_day는 거래 날짜로, 거래 시점을 알 수 있음. 이를 이용해 이번 문제를 시계열 문제로 접근할 수 있음.
- floor는 아파트 층수를 나타내는데, 아파트에서 1층은 로열층에 비해 선호되지 않는 층으로 알려져 있음. 데이터 분석을 통해 정말로 층수가 가격에 영향을 미치는지 확인해볼 수 있음.

In [11]:
train.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


3. 데이터 타입 확인

저희가 학습(train) 데이터를 이용해서 머신러닝 모델을 학습시키기 위해서는,

데이터의 타입(Dtype)이 계산 가능한 형태여야 합니다.

다음 코드를 실행해 train의 정보를 확인해 봅시다.

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216553 entries, 0 to 1216552
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1216553 non-null  int64  
 1   apartment_id            1216553 non-null  int64  
 2   city                    1216553 non-null  object 
 3   dong                    1216553 non-null  object 
 4   jibun                   1216553 non-null  object 
 5   apt                     1216553 non-null  object 
 6   addr_kr                 1216553 non-null  object 
 7   exclusive_use_area      1216553 non-null  float64
 8   year_of_completion      1216553 non-null  int64  
 9   transaction_year_month  1216553 non-null  int64  
 10  transaction_date        1216553 non-null  object 
 11  floor                   1216553 non-null  int64  
 12  transaction_real_price  1216553 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 120.7+ M

4. 데이터 통계값 확인

[문제 3]

describe()함수를 이용해 수치형 데이터의 통계값을 확인해 봅시다.

[결과 해석]
- 1사분위수 76(m^2), 2사분위수 84(m^2)로 좁은 범위에 데이터가 몰려있는 것을 확인할 수 있음. 해당 사이즈의 아파트가 매매가 활발히 이루어짐을 알 수 있음.
- 아파트 실거래가는 평균적으로 대략 15억 8천만원 정도에 표준편차가 약 7억원으로 분포가 아주 넓게 퍼져있음을 확인할 수 있음. (자세한 건 히스토그램을 통해 확인)

In [13]:
def str_to_int(string):
    if type(string) == str:
        string = string.replace(',', '')
        return int(string)
    else:
        return string

train['transaction_real_price'] = train['transaction_real_price'].apply(str_to_int)

columns = ['exclusive_use_area', 'floor', 'transaction_real_price']
train[columns].describe()

,exclusive_use_area,floor,transaction_real_price
count,1.216553e+06,1.216553e+06,1.216553e+06
mean,7.816549e+01,9.343291e+00,3.822769e+04
std,2.915113e+01,6.606500e+00,3.104898e+04
min,9.260000e+00,-4.000000e+00,1.000000e+02
25%,5.976000e+01,4.000000e+00,1.900000e+04
50%,8.241000e+01,8.000000e+00,3.090000e+04
75%,8.497000e+01,1.300000e+01,4.700000e+04
max,4.243200e+02,8.000000e+01,8.200000e+05


5. 테스트 데이터 확인

[문제 4]

빈칸을 채워 test의 인덱스 기준 상위 10개 데이터를 확인해 봅시다.

In [14]:
test.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21
5,1202911,7936,부산광역시,양정동,213-3,양정 해광빌리지,양정동 213-3 양정 해광빌리지,39.3000,2012,201706,11~20,3
6,1151450,9179,서울특별시,염창동,254-2,이너스내안에,염창동 254-2 이너스내안에,84.9100,2004,201711,1~10,13
7,1197195,4018,서울특별시,암사동,414-2,롯데캐슬퍼스트,암사동 414-2 롯데캐슬퍼스트,102.3400,2008,201712,21~31,9
8,1146078,4397,서울특별시,신정동,311,목동신시가지10,신정동 311 목동신시가지10,106.0600,1987,201712,11~20,4
9,1191026,6985,서울특별시,장지동,843,송파파인타운9단지,장지동 843 송파파인타운9단지,59.9900,2007,201712,21~31,2


6. 제출 데이터 확인

[문제 5]

빈칸을 채워 submission의 인덱스 기준 상위 10개 데이터를 확인해 봅시다.

In [15]:
submission.head(10)

,transaction_id,transaction_real_price
0,1145756,38228
1,1198704,38228
2,1222384,38228
3,1179897,38228
4,1223091,38228
5,1202911,38228
6,1151450,38228
7,1197195,38228
8,1146078,38228
9,1191026,38228


7. 타겟의 평균값 구하기

[문제 6]

아파트 실거래가의 평균 값을 구하고, 그 값을 submission에 적용해 봅시다.

In [16]:
def str_to_int(string):
    if type(string) == str:
        string = string.replace(',', '')
        return int(string)
    else:
        return string

train['transaction_real_price'] = train['transaction_real_price'].apply(str_to_int)

mean_apt_price = round(train['transaction_real_price'].mean())
submission['transaction_real_price'] = mean_apt_price
submission.head()

,transaction_id,transaction_real_price
0,1145756,38228
1,1198704,38228
2,1222384,38228
3,1179897,38228
4,1223091,38228


8. 제출 파일 생성

[문제 7]

빈칸을 채워 양식에 맞게 저희의 예측을 담은 submission을 CSV 파일로 생성한 후,

이를 제출해봅시다!

In [17]:
submission.to_csv('submission.csv', index=False)